In [1]:
import numpy as np
from pymoo.optimize import minimize
from pymoo.core.variable import Real, Integer, Choice, Binary

In [2]:
import numpy as np
from pymoo.core.problem import ElementwiseProblem

class Brt(ElementwiseProblem):
    
    def __init__(self, M=10,N=23,**kwargs):
        
        
        variables = dict()

        for dj in range(1, M*N+1):
            variables[f"x{dj:02}"] = Real(bounds=(0.0, 60.0))

        for L in range(M*N+1,2*(M*N)+1):
            variables[f"x{L:02}"] = Integer(bounds=(0, 50))
            
        for sj in range(2*(M*N)+1,3*(M*N)+1):
            variables[f"x{sj:02}"] = Integer(bounds=(0, 1))
            
        for sjk in range(3*(M*N)+1,4*(M*N)+1):
            variables[f"x{sjk:02}"] = Integer(bounds=(0, 1))
            
        for t in range(4*(M*N)+1,4*(M*N)+N+1):
            variables[f"x{t:02}"] = Real(bounds=(0, 20))#n
            
        for r in range(4*(M*N)+N+1,4*(M*N)+2*N+1):
            variables[f"x{r:02}"] = Integer(bounds=(0, 15))#n
            
        for Aj in range(4*(M*N)+2*N+1,5*(M*N)+2*N+1):
            variables[f"x{Aj:02}"] = Integer(bounds=(0, 50))
            
        for Bj in range(5*(M*N)+2*N+1,6*(M*N)+2*N+1):
            variables[f"x{Bj:02}"] = Integer(bounds=(0, 50))
            
        for Tjk in range(6*(M*N)+2*N+1,7*(M*N)+2*N+1):
            variables[f"x{Tjk:02}"] = Real(bounds=(0, 50))
            
        variables[f"x{7*(M*N)+2*N+1}"] = Integer(bounds=(0.0, M))
        variables[f"x{7*(M*N)+2*N+2}"] = Integer(bounds=(0.0, M))
        
        super().__init__(vars=variables,n_obj=2, n_constr=9,**kwargs)
        self.M=M
        self.N=N
        self.L=M*N
        self.sj=2*(M*N)
        self.sjk=3*(M*N)
        self.t=3*(M*N)+N
        self.r=3*(M*N)+2*N
        self.Aj=4*(M*N)+2*N
        self.Bj=5*(M*N)+2*N
        self.Tjk=6*(M*N)+2*N
        self.T0=30
        self.c=6
        self.Lc=50
        self.h=10
        #ljk=X[-2]
        #I=X[-1]
        
    def F1(self,x):
        ls=[]
        ls2=[]
        for j in range(self.N):
            for k in range(j+1,self.N):
                y=sum([x[self.r+j]*(x[I*self.N+j]-x[I-1*self.N+j])**2/2 for I in range(1,int(x[-2]))])
                y2=sum([x[self.Tjk+I*self.N+j]*x[self.r+j]*(x[I*self.N+j]-x[I-1*self.N+j]) for I in range(1,int(x[-2]))])
                ls.append(y)
                ls2.append(y2)
                
        return (sum(ls)+sum(ls2))
    def F2(self,x):
        ls=[]
        for i in range(self.M):
            for j in range(self.N):
                a=i*self.N+j
                y=(x[a+1]-x[a])*abs(x[self.L+a]-self.Lc)
                ls.append(y)
        return sum(ls)
     
    def ff(self,i,m,j,x):
        a=x[i*self.N+j]
        y=x[m*self.N+j]
        if a>y and (a==y or i>m):
            return 1
        elif a<y and (a==y or i<m):
            return 0
        else: return 0 
        
    def g1(self,x):
        ls=[]
        for i in range(self.M):
            y=x[i*self.N]-(i)*self.h
            ls.append(y)
        return sum(ls)
    
    def g2(self,x):
        ls=[]
        for i in range(self.M):
            for j in range(1,self.N):
                a=i*self.N+j
                y=-x[a]+x[a-1]+x[self.t+j]+(x[self.sj+a]+x[self.sj+a-1])*self.c+x[self.sj+a]*self.T0
                ls.append(y)
        return sum(ls)
    
    def g3(self,x):
        ls=[]
        for i in range(self.M):
            for j in range(self.N):
                for s in range(j+1,self.N):
                    a=i*self.N+j
                    k=i*self.N+s
                    y=-x[self.sjk+a]+x[self.sj+a]*x[self.sj+k]
                    ls.append(y)
        return sum(ls)
    
    def g4(self,x):
        ls=[]
        for i in range(self.M):
            for j in range(self.N):
                a=i*self.N+j
                ls.append(x[self.sjk+a])
        return x[-2]-sum(ls)
    
    def g5(self,x):
        ls=[]
        for i in range(self.M):
            for j in range(self.N):
                a=i*self.N+j
                s=[]
                for m in range(self.M):
                    if m==i:
                        continue
                    k=m*self.N+j
                    s.append(x[self.sjk+k]*self.ff(i,m,j,x))
                ls.append(x[self.sjk+a]*sum(s)+1)
        return x[-1]-sum(ls)
    def g7(self,x):
        ls=[]
        for I in range(int(x[-1]),int(x[-2])):
            for j in range(1,self.N):
                for s in range(j+1,self.N):
                    a=I*self.N+j
                    k=I*self.N+s
                    ls.append(x[k]-x[a]-x[self.Tjk+a]-self.T0)
        return sum(ls)
    
    def g8(self,x):
        ls=[]
        for i in range(self.M):
            for j in range(self.N):
                a=i*self.N+j
                s=[]
                for k in range(j-1):
                    K=i*self.N+k
                    y=sum([x[self.sj+K]*x[self.r+k]*(x[I*self.N+k]-x[I-1*self.N+k]) for I in range(1,int(x[-2]))])
                    s.append(y)
                ls.append(-x[self.Aj+a]+sum(s)*x[self.sj+a])
        return sum(ls)
    
    def g9(self,x):
        ls=[]
        for i in range(self.M):
            for j in range(self.N):
                a=i*self.N+j
                s=[]
                for k in range(j+1,self.N):
                    K=i*self.N+k
                    y=sum([x[self.sj+K]*x[self.r+j]*(x[I*self.N+j]-x[I-1*self.N+j]) for I in range(1,int(x[-2]))])
                    s.append(y)
                ls.append(-x[self.Bj+a]+sum(s)*x[self.sj+a])
        return sum(ls)   
    
    def g10(self,x):
        ls=[]
        for i in range(self.M):
            for j in range(1,self.N):
                a=i*self.N+j
                y=-x[self.L+a]+x[self.L+a-1]+x[self.Bj+a]-x[self.Aj+a]
                ls.append(y)
                
        return sum(ls)

    def _evaluate(self, x, out, *args, **kwargs):
        x = np.array([x[f"x{k:02}"] for k in range(1,7*(self.M*self.N)+2*self.N+3)])
        # objectives
        
        f2=self.F2(x)
        f1=self.F1(x)
        out["F"] = [f1,f2]
        
        # constraints
        g1=self.g1(x)
        g2=self.g2(x)
        g3=self.g3(x)
        g4=self.g4(x)
        g5=self.g5(x)
        g7=self.g7(x)
        g8=self.g8(x)
        g9=self.g9(x)
        g10=self.g10(x)
        out["G"] =[g1,g2,g3,g4,g5,g7,g8,g9,g10]

problem = Brt()
for name, var in problem.vars.items():
    print(name, var)

x01 <pymoo.core.variable.Real object at 0x00000179E886B550>
x02 <pymoo.core.variable.Real object at 0x00000179CFA4F370>
x03 <pymoo.core.variable.Real object at 0x00000179CFA4F3D0>
x04 <pymoo.core.variable.Real object at 0x00000179CFA4F430>
x05 <pymoo.core.variable.Real object at 0x00000179CFA4F490>
x06 <pymoo.core.variable.Real object at 0x00000179CFA4F4F0>
x07 <pymoo.core.variable.Real object at 0x00000179CFA4F550>
x08 <pymoo.core.variable.Real object at 0x00000179CFA4F5B0>
x09 <pymoo.core.variable.Real object at 0x00000179CFA4F610>
x10 <pymoo.core.variable.Real object at 0x00000179CFA4F670>
x11 <pymoo.core.variable.Real object at 0x00000179CFA4F6D0>
x12 <pymoo.core.variable.Real object at 0x00000179CFA4F730>
x13 <pymoo.core.variable.Real object at 0x00000179CFA4F790>
x14 <pymoo.core.variable.Real object at 0x00000179CFA4F7F0>
x15 <pymoo.core.variable.Real object at 0x00000179CFA4F850>
x16 <pymoo.core.variable.Real object at 0x00000179CFA4F8B0>
x17 <pymoo.core.variable.Real object at 

In [ ]:
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.termination import get_termination
from pymoo.core.mixed import MixedVariableMating, MixedVariableGA, MixedVariableSampling, MixedVariableDuplicateElimination
from pymoo.visualization.scatter import Scatter

algorithm = NSGA2(pop_size=200,
                  sampling=MixedVariableSampling(),
                  mating=MixedVariableMating(eliminate_duplicates=MixedVariableDuplicateElimination()),
                  eliminate_duplicates=MixedVariableDuplicateElimination(),
                  )
termination = get_termination("n_gen", 500)

res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               verbose=True)


n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |      200 |      1 |  4.186946E+03 |  1.190742E+04 |             - |             -
     2 |      400 |      1 |  3.199563E+03 |  4.555260E+03 |             - |             -
     3 |      600 |      1 |  2.201532E+03 |  4.001419E+03 |             - |             -
     4 |      800 |      1 |  1.892586E+03 |  3.372757E+03 |             - |             -
     5 |     1000 |      1 |  1.352698E+03 |  2.822806E+03 |             - |             -
     6 |     1200 |      1 |  1.178069E+03 |  2.306682E+03 |             - |             -
     7 |     1400 |      1 |  6.075812E+02 |  1.834700E+03 |             - |             -
     8 |     1600 |      1 |  3.172135E+02 |  1.420334E+03 |             - |             -
     9 |     1800 |      1 |  1.470135E+02 |  1.063005E+03 |             - |             -
    10 |     2000 |      1 |  1.269577E+02 |  7.722226E+02 |             - |             -

    89 |    17800 |      4 |  0.000000E+00 |  0.000000E+00 |  0.1224155646 |             f
    90 |    18000 |      9 |  0.000000E+00 |  0.000000E+00 |  0.0493905152 |         ideal
    91 |    18200 |      9 |  0.000000E+00 |  0.000000E+00 |  0.0774708984 |         ideal
    92 |    18400 |     12 |  0.000000E+00 |  0.000000E+00 |  0.0067142874 |         ideal
    93 |    18600 |     14 |  0.000000E+00 |  0.000000E+00 |  0.0601473932 |         ideal
    94 |    18800 |     10 |  0.000000E+00 |  0.000000E+00 |  0.2287154217 |         ideal
    95 |    19000 |     10 |  0.000000E+00 |  0.000000E+00 |  0.0704670997 |             f
    96 |    19200 |     13 |  0.000000E+00 |  0.000000E+00 |  0.1464411819 |         ideal
    97 |    19400 |      9 |  0.000000E+00 |  0.000000E+00 |  0.0026542417 |         ideal
    98 |    19600 |      9 |  0.000000E+00 |  0.000000E+00 |  0.0351159462 |         ideal
    99 |    19800 |     13 |  0.000000E+00 |  0.000000E+00 |  0.0356086774 |         ideal

In [ ]:
plot = Scatter()
plot.add(problem.pareto_front(), plot_type="line", color="black", alpha=0.7)
plot.add(res.F, facecolor="none", edgecolor="red")
plot.show()

In [14]:
variabl = dict()
for dj in range(1, 20+1):
	variabl[f"x{dj:04}"] = Real(bounds=(0.0, 60.0))

In [15]:
variabl

{'x0001': <pymoo.core.variable.Real at 0x1e29e17d720>,
 'x0002': <pymoo.core.variable.Real at 0x1e29e17ea70>,
 'x0003': <pymoo.core.variable.Real at 0x1e29e17d750>,
 'x0004': <pymoo.core.variable.Real at 0x1e29e17de70>,
 'x0005': <pymoo.core.variable.Real at 0x1e29e17db40>,
 'x0006': <pymoo.core.variable.Real at 0x1e29e17dc00>,
 'x0007': <pymoo.core.variable.Real at 0x1e29e17d7e0>,
 'x0008': <pymoo.core.variable.Real at 0x1e29e17de10>,
 'x0009': <pymoo.core.variable.Real at 0x1e29e17dde0>,
 'x0010': <pymoo.core.variable.Real at 0x1e29e17ddb0>,
 'x0011': <pymoo.core.variable.Real at 0x1e29e17d630>,
 'x0012': <pymoo.core.variable.Real at 0x1e29e17e3e0>,
 'x0013': <pymoo.core.variable.Real at 0x1e29e17d6f0>,
 'x0014': <pymoo.core.variable.Real at 0x1e29e17d5a0>,
 'x0015': <pymoo.core.variable.Real at 0x1e29e17d5d0>,
 'x0016': <pymoo.core.variable.Real at 0x1e29e17e2f0>,
 'x0017': <pymoo.core.variable.Real at 0x1e29e17dc30>,
 'x0018': <pymoo.core.variable.Real at 0x1e29e17dd20>,
 'x0019': 